<a href="https://colab.research.google.com/github/surajghuwalewala/CE888_Data_Science_and_Decision_Making/blob/master/Lab_5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender System

In [0]:
import numpy as np
import pandas as pd

## Loading the jester dataset
- Each row corresponds to a user.
- First column is total number of jokes rated by that user
- The shape of this data is 24983 x 101
- 24983 users have rated 100 jokes
- Columns 1 to 100 are ratings for each joke in the range of -10 to 10.
- Value of 99 indicates no rating was given.

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", header=None)
df.shape

(24983, 101)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(orig!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
## Replacing 10% of values to 99
val_set, idx = replace(df.values, 0.1)

In [0]:
def latentFactorModelling(data, n_latent_factors = 2, alpha = 0.0001, iterations=300):

  user_ratings = data.values
  # Initialise as random values
  latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
  latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
  
  
  def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


  def train(user_id, item_id, rating, alpha = alpha):
      
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
      

  def sgd(iterations=iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating != 99.0):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 5) == 0:
            print("MSE - Iteration {} : {}".format(iteration, mse))

  sgd(iterations)
    
  return latent_user_preferences, latent_item_features

In [7]:
latent_user_preferences, latent_item_features = latentFactorModelling(df, iterations=25,  n_latent_factors = 3)

MSE - Iteration 0 : 51.24881353294431
MSE - Iteration 5 : 24.64058470537922
MSE - Iteration 10 : 24.623013253636824
MSE - Iteration 15 : 24.57623533613078
MSE - Iteration 20 : 24.101029782724172


In [8]:
print("latent_user_preferences: ",latent_user_preferences.shape)
print("latent_item_features: ",latent_item_features.shape)

latent_user_preferences:  (24983, 3)
latent_item_features:  (101, 3)


In [0]:
## Train predictions
predictions = latent_user_preferences.dot(latent_item_features.T)

In [0]:
def calculate_mse(orig, pred):
  error = []
  for user_id in range(orig.shape[0]):
    for item_id in range(orig.shape[1]):
      user_rating = orig[user_id][item_id]
      if(user_rating!=99.0):
        err = pred[user_id][item_id] - user_rating
        error.append(err)

  mse = (np.array(error) ** 2).mean() 

  return mse

In [11]:
## Training Mean Square Error

mse = calculate_mse(df.values, predictions)

print("Training MSE: ", mse)

Training MSE:  21.64116053145564


In [12]:
df.values.shape

(24983, 101)

In [13]:
## Validation predictions

val_mse = calculate_mse(val_set, predictions)

print("Validation MSE: ", val_mse)


Validation MSE:  21.64435494287914


Creating different validations sets and calculating MSE for each to calculate a mean mse

In [14]:
val_mse_arr = []
for itr in range(10):
    val_set, idx = replace(df.values, 0.1)  
    val_mse = calculate_mse(val_set, predictions)
    print("Iteration {} - Validation MSE: {}".format(itr+1, val_mse))
    val_mse_arr.append(val_mse)

mean_val_mse = np.mean(val_mse_arr)
print("Mean validation MSE is {}".format(mean_val_mse))




Iteration 1 - Validation MSE: 21.635219565628
Iteration 2 - Validation MSE: 21.647657410330435
Iteration 3 - Validation MSE: 21.63855977784908
Iteration 4 - Validation MSE: 21.65019556569118
Iteration 5 - Validation MSE: 21.63740243054737
Iteration 6 - Validation MSE: 21.640726649430675
Iteration 7 - Validation MSE: 21.642592395914214
Iteration 8 - Validation MSE: 21.638407335787818
Iteration 9 - Validation MSE: 21.631932395855607
Iteration 10 - Validation MSE: 21.65129901346635
Mean validation MSE is 21.64139925405007


Tried running the cells below several times at maximum RAM capacity of colab, but always ran into memory error. 


In [0]:
values = [zip(df.values[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns


In [0]:
comparison_data